# Polarizability 2018

In [ ]:
import pandas as pd

You can get the polarizability table as csv from [this link](https://www.tandfonline.com/action/downloadTable?id=T0001&doi=10.1080%2F00268976.2018.1535143&downloadType=CSV).

In [ ]:
df = pd.read_csv('data/T0001-10.1080%2F00268976.2018.1535143.csv')

In [ ]:
df.info()

In [ ]:
df.head()

Find empty values in the csv, replace the with `None` and forward fill the last value to complete the table.

In [ ]:
mask = ~df['Z'].str.isdigit()

In [ ]:
df.loc[mask, 'Z'] = None
df.loc[mask, 'Atom'] = None

df.loc[:, 'Z'] = df['Z'].ffill()
df.loc[:, 'Atom'] = df['Atom'].ffill()

In [ ]:
df.head()

We will only need the recommended values and their uncertainties.

In [ ]:
df[df['Comments'] == 'recommended']

Extract the numerical values and the errors from the strings.

In [ ]:
df.loc[:, 'αD value'] = df['αD'].str.extract(r'^(\d+\.\d+|\d+)', expand=False).astype(float)
df.loc[:, 'αD error'] = df['αD'].str.extract(r'±\s*(\d+\.\d+|\d+)', expand=False).astype(float)

In [ ]:
df.head(15)

## Update the polarizability values in the `elements.db`

In [ ]:
from mendeleev import get_session, Element
session = get_session()

In [ ]:
alpha = df[df['Comments'] == 'recommended']

In [ ]:
alpha.info()

Check if there are any elements multiply defined

In [ ]:
(alpha.Z.value_counts() > 1).sum()

In [ ]:
for i, row in alpha.iterrows():
    if int(row['Z']) <= 118:
        e = session.query(Element).filter(Element.atomic_number == int(row['Z'])).one()
    
        print(e.symbol, e.dipole_polarizability, e.dipole_polarizability_unc)
        print(row['αD value'], row['αD error'])
        e.dipole_polarizability = row['αD value']
        e.dipole_polarizability_unc = row['αD error']
        session.add(e)
session.commit()

In [ ]:
session.close()